In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Sentiment Analysis of Amazon Reviews\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Sentiment Analysis of Amazon Reviews'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PreprocessingConfig:
    root_dir: Path
    ingestion_dir: Path
    output_dir: Path

In [6]:

from pathlib import Path
from dataclasses import dataclass
import yaml
from sentimentanalyzer.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from sentimentanalyzer.utils.common import create_directories,read_yaml

In [7]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

  

    def get_preprocessing_config(self) -> PreprocessingConfig:
        config = self.config.data_preprocessing

        create_directories([config.root_dir])

        data_preprocessing_config = PreprocessingConfig(
                      root_dir =config.root_dir,
                      ingestion_dir = config.ingestion_dir,
                      output_dir = config.output_dir
               )
        
        return data_preprocessing_config

In [8]:
import pandas as pd
import numpy as np
from pathlib import Path
from src.sentimentanalyzer.utils.common import convert_to_csv, preprocess_review_list, clean_sentiment_text, get_labels_and_texts_from_txt
import re, contractions, emoji
from sentimentanalyzer.logging import logger


In [9]:
import numpy as np
from pathlib import Path
from typing import List, Tuple

class DataPreprocessing:
    def __init__(self, config: PreprocessingConfig):
        self.config = config

        # Ensure output directory exists
        create_directories([Path(self.config.root_dir)])

        ingestion_dir = Path(self.config.ingestion_dir)
        output_dir = Path(self.config.output_dir)
        output_dir.mkdir(parents=True, exist_ok=True)

        datasets = {
            "train": ingestion_dir / "train.ft.txt",
            "test": ingestion_dir / "test.ft.txt"
        }

        for split, filepath in datasets.items():
            if filepath.exists():
                labels, texts = get_labels_and_texts_from_txt(filepath, max_lines=100)

                # Save with split in filename
                labels_path = output_dir / f"{split}_labels.npy"
                texts_path = output_dir / f"{split}_texts.txt"

                np.save(labels_path, labels)
                with open(texts_path, "w", encoding="utf-8") as f_out:
                    for text in texts:
                        f_out.write(text + "\n")

                print(f"{split.capitalize()} set saved:")
                print(f"✅ Labels -> {labels_path}")
                print(f"✅ Texts  -> {texts_path}")
            else:
                print(f"⚠️ No file found for {split} at {filepath}")


In [10]:

try:
    config = ConfigurationManager()
    data_preprocessing_config = config.get_preprocessing_config()
    data_preprocessor = DataPreprocessing(config=data_preprocessing_config)
except Exception as e:
    raise e


[2025-06-17 19:33:21,516: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-17 19:33:21,518: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-17 19:33:21,527: INFO: common: created directory at: artifacts]
[2025-06-17 19:33:21,529: INFO: common: created directory at: artifacts/data_preprocessing]
[2025-06-17 19:33:21,532: INFO: common: created directory at: artifacts\data_preprocessing]
Train set saved:
✅ Labels -> artifacts\data_preprocessing\train_labels.npy
✅ Texts  -> artifacts\data_preprocessing\train_texts.txt
Test set saved:
✅ Labels -> artifacts\data_preprocessing\test_labels.npy
✅ Texts  -> artifacts\data_preprocessing\test_texts.txt
